In [1]:
! pip install import-ipynb

/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


In [2]:
from model import BodyposeModel
import cv2
import numpy as np
import math
import time
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
import matplotlib
import torch
from torchvision import transforms
import util
from torchsummary import summary
from body import Body
import import_ipynb
from train_model import BodyposeBackbone, MainModel
from coco_dataset import COCODataset, transform_image
from copy import deepcopy

/tmp/ipykernel_32778/2162950386.py:6: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


importing Jupyter notebook from train_model.ipynb
importing Jupyter notebook from coco_dataset.ipynb


In [3]:
NO_OF_OUTPUTS = 21

In [26]:
colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
            [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
            [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85]]

In [4]:
backbone_model_path = '../model/body_pose_model.pth'

In [5]:
backbone_model = BodyposeBackbone()

In [6]:
backbone_params = torch.load(backbone_model_path)

In [7]:
backbone_dict = util.transfer(backbone_model, backbone_params)

In [8]:
backbone_model.load_state_dict(backbone_dict)

<All keys matched successfully>

In [9]:
main_model = MainModel(NO_OF_OUTPUTS)

In [10]:
train_data = COCODataset('/media/jakub/One Touch/coco_pose/coco2017labels-pose/coco-pose', transforms=transform_image(split='train'))

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [14]:
anns = []
cats = []


In [15]:
skeleton = train_data.coco.cats[1]['skeleton']
skeleton = [[connection[0]-1, connection[1]-1] for connection in skeleton]

In [16]:
for id in train_data.coco.getAnnIds():
    if train_data.coco.anns[id]['image_id'] == train_data.coco.getImgIds()[0]:
        ann = train_data.coco.anns[id]
        anns.append(ann)


In [18]:
bbox = anns[0]['bbox']

In [19]:
keypoints = anns[0]['keypoints']

In [20]:
keypoints = [[keypoints[i], keypoints[i+1], keypoints[i+2]] for i in range(0, len(keypoints), 3)]

In [21]:
bbox = [int(box) for box in bbox]

In [27]:
image = train_data._load_image(train_data.coco.getImgIds()[0])
cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[0]+bbox[2], bbox[1]+bbox[3]), (0, 0, 255))
for i, connection in enumerate(skeleton):
    point1 = keypoints[connection[0]]
    point2 = keypoints[connection[1]]
    if point1[2] != 0 and point2[2] != 0:
        cv2.line(image, point1[:-1], point2[:-1], color=colors[i], thickness=3)
cv2.imshow('Example image', image)
cv2.waitKey(5_000)
cv2.destroyAllWindows()

In [23]:
def train(backbone_model, main_model, loss, optimizer, inputs, outputs):
    optimizer.zero_grad()
    main_model.train()

    out1 = backbone_model(inputs)
    logits = main_model(out1)
    output = loss.forward(logits, outputs)
    output.backward()
    optimizer.step()

    return output.item()

In [24]:
loss = torch.nn.MSELoss()
params = [p for p in main_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params)

In [25]:
epochs = 10
batch_size = 4
num_of_batches = Xtrain_torch.size[0] // batch_size

for epoch in range(epochs):
    for batch in range(num_of_batches):
        inputs = Xtrain_torch[batch*batch_size:(batch+1)*batch_size]
        outputs = Ytrain_torch[batch*batch_size:(batch+1)*batch_size]
        output = train(backbone_model, main_model, loss, optimizer, inputs, outputs)

NameError: name 'Xtrain_torch' is not defined